#### Network Flow Application: Maximum Bipartite Matching


In [5]:
# use BFS to find path from s to t in Gf
def find_st_path(adjacency_list, s, t):
    assert (s >= 0 and s < len(adjacency_list)), "s is out of range"
    # initialize all vertices as undiscovered except for s
    discovered = [False] * len(adjacency_list)
    discovered[s] = True
    parent = [None] * len(adjacency_list)
    # intiialize BFS queue
    Q = []
    Q.append(s)
    # initialize s-t path
    path = []
    # run BFS
    while len(Q)>0:
        # pop vertex from front of queue
        u = Q.pop(0)
        # find vertices adjacent to u and add them to the Q if they are undiscovered
        for (v,ce) in adjacency_list[u]:
            if not discovered[v]:
                discovered[v] = True
                parent[v] = (u, ce)
                # if v is t, then we have found the path
                if v == t:
                    # reconstruct path from s to t following parent pointers
                    while parent[t] != None:
                        # add edge to path
                        edge = (parent[t][0], t, parent[t][1])
                        path.append(edge)
                        t = parent[t][0]
                    return path
                # add to Q
                Q.append(v)
 
    # if we reach here, then there is no s-t path
    return path            

def construct_residual_graph(adjacency_list, flow, capacity):
    Gf_adjacency_list = {u:[] for u in adjacency_list.keys()}
    for u in range(len(adjacency_list)):
        for (v, ce) in adjacency_list[u]:
            # add forward edge if there's non-zero residual capacity
            residual_ce = capacity[(u, v)] - flow[(u, v)]
            if residual_ce > 0:
                Gf_adjacency_list[u].append((v, residual_ce))
            # add backward edge if there's non-zero flow 
            if flow[(u, v)] > 0:
                Gf_adjacency_list[v].append((u, flow[(u, v)]))

    return Gf_adjacency_list            

def FF_nosclaing(adjacency_list, verbose=False):
    # initialize the flow to 0
    flow = {}
    capacity = {}
    for u in adjacency_list.keys():
        for (v, ce) in adjacency_list[u]:
            flow[(u, v)] = 0
            capacity[(u, v)] = ce

    # construct residual graph
    Gf_adjacency_list = construct_residual_graph(adjacency_list, flow, capacity)
    if verbose:
        print("Initial residual graph:")
        print(Gf_adjacency_list)

    s = 0
    t = len(adjacency_list)-1 
    num_iterations = 0
    # run augmenting iterations
    while True:
        # find s-t path in Gf
        path = find_st_path(Gf_adjacency_list, s, t)
        # if there is no s-t path, then we are done
        if len(path) == 0:
            if verbose:
                flow_value = sum([flow[(s, v)] for (v, _) in adjacency_list[s]])
                print(f"\nMax flow found! |f| = {flow_value}, total iterations = {num_iterations}")
            return flow
        else:
            # find bottleneck capacity of path
            bottleneck_capacity = float('inf')
            for (u, v, ce) in path:
                if ce < bottleneck_capacity:
                    bottleneck_capacity = ce
            # augment flow along path
            for (u, v, ce) in path:
                if (u, v) in capacity:
                    # increase flow along forward edge (u, v)
                    flow[(u, v)] += bottleneck_capacity
                else:
                    # decrease flow along backward edge (v, u)
                    flow[(v, u)] -= bottleneck_capacity 
            # construct updated residual graph        
            Gf_adjacency_list = construct_residual_graph(adjacency_list, flow, capacity)     
            
            if verbose:
                print(f"\nFlow augmented along path: {path} by amount: {bottleneck_capacity}")
                print(f"New residual graph: \n {Gf_adjacency_list}")
                print(f"Augmented Flow: \n {flow}")   

            num_iterations += 1        

# assume that the input graph is bipartite and represented as an adjacency list, and vertices in X and Y are numbered from 1 to n
def create_equivalent_flow_network(adjacency_list, X, Y):
    # create artificial source and sink vertices 
    s = 0
    t = len(adjacency_list)+1
    # create equivalent flow network adjacency list (all edges will have capacity 1)
    G = {u:[] for u in range(len(adjacency_list)+2)}
    # connect source to vertices in X
    for u in X:
        G[s].append((u, 1))
    # connect vertices in Y to sink
    for v in Y:
        G[v].append((t, 1))
    # connect vertices in X to vertices in Y
    for u in X:
        for v in adjacency_list[u]:
            G[u].append((v, 1))        
   
    return G

def maximum_bipartite_matching(adjacency_list, X, Y, verbose=False):
    # create flow network
    G = create_equivalent_flow_network(adjacency_list, X, Y)
    if verbose:
        print("Equivalent flow network:")
        print(G)
    # run FF algorithm to get max flow
    flow = FF_nosclaing(G)
    # extract maximum matching from flow
    matching = []
    for (u,v), f in flow.items():
        if u in X and v in Y and f > 0:
            matching.append((u,v))

    if verbose:
        print(f"Maximum matching: {matching}")
    return matching                


In [6]:
adjacency_list = {1:[6,7], 2:[6], 3:[6,8], 4:[8,9,10], 5:[9,10], 6:[1,2,3], 7:[1], 8:[3,4], 9:[4,5], 10:[4,5]}
X = {1,2,3,4,5}
Y = {6,7,8,9,10}

maximum_bipartite_matching(adjacency_list, X, Y, verbose=True)

Equivalent flow network:
{0: [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], 1: [(6, 1), (7, 1)], 2: [(6, 1)], 3: [(6, 1), (8, 1)], 4: [(8, 1), (9, 1), (10, 1)], 5: [(9, 1), (10, 1)], 6: [(11, 1)], 7: [(11, 1)], 8: [(11, 1)], 9: [(11, 1)], 10: [(11, 1)], 11: []}
Maximum matching: [(1, 7), (2, 6), (3, 8), (4, 9), (5, 10)]


[(1, 7), (2, 6), (3, 8), (4, 9), (5, 10)]